In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, LSTM

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential

from datetime import datetime,date, timedelta

import math
 




In [2]:
KODE = 'TLKM.JK'
today_date = date.today()
td = timedelta(366)
mulai = today_date - td
end_ = today_date + timedelta(1)

import yfinance as yf

data = yf.download(KODE, start=mulai, end = end_)
datam = yf.download(KODE, period="max", auto_adjust=True)

data.tail()
#datam.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-05-23 00:00:00+07:00,4030.0,4080.0,4030.0,4040.0,4040.0,132751000
2023-05-24 00:00:00+07:00,4020.0,4200.0,4020.0,4160.0,4160.0,194127500
2023-05-25 00:00:00+07:00,4190.0,4220.0,4120.0,4120.0,4120.0,122121000
2023-05-26 00:00:00+07:00,4140.0,4160.0,4110.0,4130.0,4130.0,62211800
2023-05-29 00:00:00+07:00,4120.0,4200.0,4090.0,4200.0,4200.0,77540300


In [3]:
data_close = data['Close']
data_close

Date
2022-05-30 00:00:00+07:00    4250.0
2022-05-31 00:00:00+07:00    4310.0
2022-06-02 00:00:00+07:00    4290.0
2022-06-03 00:00:00+07:00    4310.0
2022-06-06 00:00:00+07:00    4340.0
                              ...  
2023-05-23 00:00:00+07:00    4040.0
2023-05-24 00:00:00+07:00    4160.0
2023-05-25 00:00:00+07:00    4120.0
2023-05-26 00:00:00+07:00    4130.0
2023-05-29 00:00:00+07:00    4200.0
Name: Close, Length: 248, dtype: float64

df_log = np.log(data_close)
df_log

train_data, test_data = df_log[3:int(len(df_log)*0.9)], df_log[int(len(df_log)*0.9):]
len(test_data)

In [4]:
# Pra-pemrosesan data
scaler = MinMaxScaler(feature_range=(0, 1))
data_close = scaler.fit_transform(np.array(data['Close']).reshape(-1, 1))
data_close


array([[0.56521739],
       [0.6173913 ],
       [0.6       ],
       [0.6173913 ],
       [0.64347826],
       [0.56521739],
       [0.43478261],
       [0.39130435],
       [0.36521739],
       [0.4173913 ],
       [0.44347826],
       [0.4       ],
       [0.3826087 ],
       [0.45217391],
       [0.3826087 ],
       [0.44347826],
       [0.39130435],
       [0.48695652],
       [0.47826087],
       [0.40869565],
       [0.37391304],
       [0.3826087 ],
       [0.34782609],
       [0.36521739],
       [0.33913043],
       [0.33913043],
       [0.36521739],
       [0.3826087 ],
       [0.35652174],
       [0.3826087 ],
       [0.35652174],
       [0.29565217],
       [0.36521739],
       [0.47826087],
       [0.51304348],
       [0.48695652],
       [0.53043478],
       [0.57391304],
       [0.54782609],
       [0.55652174],
       [0.56521739],
       [0.60869565],
       [0.55652174],
       [0.54782609],
       [0.60869565],
       [0.73913043],
       [0.7826087 ],
       [0.826

In [5]:
# Bagi data menjadi data pelatihan dan pengujian
train_size = int(len(data_close) * 0.9)
test_size = len(data_close) - train_size
train_data, test_data = data_close[0:train_size,:], data_close[train_size:len(data_close),:]

In [6]:
# Buat urutan data pelatihan dan pengujian
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [7]:
look_back = 60
train_X, train_Y = create_dataset(train_data, look_back)
test_X, test_Y = create_dataset(test_data, look_back)

In [8]:
# Definisikan model bi-LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(look_back, 1)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(1))

In [9]:
# Lakukan pelatihan model
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(train_X, train_Y, epochs=100, batch_size=32, validation_split=0.1)


Epoch 1/100
5/5 [==============================] - 6s 280ms/step - loss: 0.1050 - val_loss: 0.0124
Epoch 2/100
5/5 [==============================] - 0s 35ms/step - loss: 0.0437 - val_loss: 0.0248
Epoch 3/100
5/5 [==============================] - 0s 36ms/step - loss: 0.0241 - val_loss: 0.0373
Epoch 4/100
5/5 [==============================] - 0s 35ms/step - loss: 0.0179 - val_loss: 0.0145
Epoch 5/100
5/5 [==============================] - 0s 34ms/step - loss: 0.0134 - val_loss: 0.0112
Epoch 6/100
5/5 [==============================] - 0s 38ms/step - loss: 0.0090 - val_loss: 0.0243
Epoch 7/100
5/5 [==============================] - 0s 37ms/step - loss: 0.0105 - val_loss: 0.0129
Epoch 8/100
5/5 [==============================] - 0s 37ms/step - loss: 0.0091 - val_loss: 0.0098
Epoch 9/100
5/5 [==============================] - 0s 34ms/step - loss: 0.0076 - val_loss: 0.0165
Epoch 10/100
5/5 [==============================] - 0s 38ms/step - loss: 0.0075 - val_loss: 0.0085
Epoch 11/100
5/5 [

Epoch 84/100
5/5 [==============================] - 0s 35ms/step - loss: 0.0048 - val_loss: 0.0045
Epoch 85/100
5/5 [==============================] - 0s 33ms/step - loss: 0.0047 - val_loss: 0.0077
Epoch 86/100
5/5 [==============================] - 0s 33ms/step - loss: 0.0051 - val_loss: 0.0044
Epoch 87/100
5/5 [==============================] - 0s 34ms/step - loss: 0.0058 - val_loss: 0.0073
Epoch 88/100
5/5 [==============================] - 0s 34ms/step - loss: 0.0062 - val_loss: 0.0057
Epoch 89/100
5/5 [==============================] - 0s 35ms/step - loss: 0.0053 - val_loss: 0.0069
Epoch 90/100
5/5 [==============================] - 0s 33ms/step - loss: 0.0048 - val_loss: 0.0054
Epoch 91/100
5/5 [==============================] - 0s 32ms/step - loss: 0.0050 - val_loss: 0.0074
Epoch 92/100
5/5 [==============================] - 0s 33ms/step - loss: 0.0046 - val_loss: 0.0059
Epoch 93/100
5/5 [==============================] - 0s 34ms/step - loss: 0.0049 - val_loss: 0.0068
Epoch 94/1

In [10]:
# Lakukan evaluasi model
train_score = model.evaluate(train_X, train_Y, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (train_score, np.sqrt(train_score)))
test_score = model.evaluate(test_X, test_Y, verbose=0)
#print('Test Score: %.2f MSE (%.2f RMSE)' % (test_score, np.sqrt(test_score)))

Train Score: 0.00 MSE (0.06 RMSE)


In [11]:
# Lakukan prediksi harga saham Telkom
predicted_price = model.predict(test_X)
predicted_price = scaler.inverse_transform(predicted_price)

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.